In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 14)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-14
yesterday: 2023-07-13


#### Restart and Run All

In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1280043,TTB,2023-07-13,2023-07-13,0,1.52,1.53,0.01,0.66%,1,0,1.58,1.52,"7,508,010,136",2023-07-13,2023-07-13,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'TTB'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9717,TTB,0.00,0.00,O,1.09,0.00,0.00,0.00,0.00,"20,000",-9,17,0,0,0,52WL,SET50


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-07-13'
ORDER BY a.name



,name
0,AH
1,AWC
2,BBL
3,BCH
4,BDMS
5,BEM
6,BH
7,CPNREIT
8,DIF
9,EA


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-07-13' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,CPNREIT,2023-07-13,2023-07-13,10.90,10.80,"15,892,330",11.00,10.70,B,SET,-0.10,-0.92%
1,DIF,2023-07-04,2023-07-13,11.00,10.70,"561,222,064",11.20,10.70,B,SET,-0.30,-2.73%
2,GVREIT,2023-07-12,2023-07-13,8.25,8.30,"1,229,618",8.50,8.25,B,SET,0.05,0.61%
3,JMT,2023-07-13,2023-07-13,33.00,34.75,"91,476,376",37.25,33.00,B,SET50,1.75,5.30%
4,MCS,2023-07-06,2023-07-13,6.45,6.20,"14,584,016",6.65,6.20,B,SET,-0.25,-3.88%
5,SENA,2023-07-04,2023-07-13,3.00,2.98,"21,091,856",3.02,2.94,B,SET,-0.02,-0.67%
6,SINGER,2023-07-13,2023-07-13,8.75,8.85,"161,377,952",9.80,8.75,B,SET100,0.10,1.14%
7,STA,2023-07-13,2023-07-13,16.60,17.00,"8,877,496",17.40,16.60,B,SET100,0.40,2.41%
8,SYNEX,2023-07-12,2023-07-13,12.10,11.70,"1,968,720",12.10,11.70,B,SET,-0.40,-3.31%
9,TMT,2023-07-13,2023-07-13,7.25,7.30,"1,227,936",7.50,7.25,B,SET,0.05,0.69%


In [11]:
df.shape

(35, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,CPNREIT,2023-07-13,2023-07-13,10.90,10.80,"15,892,330",11.00,10.70,-0.92%,B
1,DIF,2023-07-04,2023-07-13,11.00,10.70,"561,222,064",11.20,10.70,-2.73%,B
2,GVREIT,2023-07-12,2023-07-13,8.25,8.30,"1,229,618",8.50,8.25,0.61%,B
3,JMT,2023-07-13,2023-07-13,33.00,34.75,"91,476,376",37.25,33.00,5.30%,B
4,MCS,2023-07-06,2023-07-13,6.45,6.20,"14,584,016",6.65,6.20,-3.88%,B
5,SENA,2023-07-04,2023-07-13,3.00,2.98,"21,091,856",3.02,2.94,-0.67%,B
6,SINGER,2023-07-13,2023-07-13,8.75,8.85,"161,377,952",9.80,8.75,1.14%,B
7,STA,2023-07-13,2023-07-13,16.60,17.00,"8,877,496",17.40,16.60,2.41%,B
8,SYNEX,2023-07-12,2023-07-13,12.10,11.70,"1,968,720",12.10,11.70,-3.31%,B
9,TMT,2023-07-13,2023-07-13,7.25,7.30,"1,227,936",7.50,7.25,0.69%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(35, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,CPNREIT,2023-07-13,2023-07-13,10.90,10.80,15892330,11.00,10.70,-0.92,B
1,DIF,2023-07-04,2023-07-13,11.00,10.70,561222064,11.20,10.70,-2.73,B
2,GVREIT,2023-07-12,2023-07-13,8.25,8.30,1229618,8.50,8.25,0.61,B
3,JMT,2023-07-13,2023-07-13,33.00,34.75,91476376,37.25,33.00,5.30,B
4,MCS,2023-07-06,2023-07-13,6.45,6.20,14584016,6.65,6.20,-3.88,B
5,SENA,2023-07-04,2023-07-13,3.00,2.98,21091856,3.02,2.94,-0.67,B
6,SINGER,2023-07-13,2023-07-13,8.75,8.85,161377952,9.80,8.75,1.14,B
7,STA,2023-07-13,2023-07-13,16.60,17.00,8877496,17.40,16.60,2.41,B
8,SYNEX,2023-07-12,2023-07-13,12.10,11.70,1968720,12.10,11.70,-3.31,B
9,TMT,2023-07-13,2023-07-13,7.25,7.30,1227936,7.50,7.25,0.69,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-07-14,1.84,0.04,2.222222,1.80,1.85,1.80,18372807,3.356103e+04
1,ADVANC,2023-07-14,222.00,4.00,1.834862,219.00,222.00,218.00,6238281,1.377003e+06
2,AEONTS,2023-07-14,164.00,1.50,0.923077,163.00,168.00,162.00,862765,1.418557e+05
3,AH,2023-07-14,34.00,0.50,1.492537,33.75,34.50,33.25,1150979,3.894899e+04
4,AIMIRT,2023-07-14,11.50,0.10,0.877193,11.40,11.50,11.40,68400,7.798800e+02
...,...,...,...,...,...,...,...,...,...,...
182,WHAIR,2023-07-14,7.05,0.00,0.000000,7.05,7.05,7.00,153401,1.079412e+03
183,WHART,2023-07-14,10.70,0.10,0.943396,10.60,10.70,10.60,1013328,1.083450e+04
184,WHAUP,2023-07-14,3.56,0.00,0.000000,3.54,3.58,3.54,640805,2.283290e+03
185,WICE,2023-07-14,8.55,0.25,3.012048,8.35,8.55,8.35,516007,4.377220e+03


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,CPNREIT,2023-07-13,2023-07-13,10.90,10.80,15892330,11.00,10.70,-0.92,B,2023-07-14,10.90,0.10,0.925926,10.90,10.90,10.80,1206570,1.315106e+04
1,DIF,2023-07-04,2023-07-13,11.00,10.70,561222064,11.20,10.70,-2.73,B,2023-07-14,10.70,0.00,0.000000,10.60,10.80,10.50,10919380,1.162411e+05
2,GVREIT,2023-07-12,2023-07-13,8.25,8.30,1229618,8.50,8.25,0.61,B,2023-07-14,8.40,0.10,1.204819,8.35,8.45,8.30,603905,5.068887e+03
3,JMT,2023-07-13,2023-07-13,33.00,34.75,91476376,37.25,33.00,5.30,B,2023-07-14,35.50,0.75,2.158273,34.50,35.75,34.25,20654942,7.244717e+05
4,MCS,2023-07-06,2023-07-13,6.45,6.20,14584016,6.65,6.20,-3.88,B,2023-07-14,6.35,0.15,2.419355,6.20,6.35,6.20,135358,8.499632e+02
5,SENA,2023-07-04,2023-07-13,3.00,2.98,21091856,3.02,2.94,-0.67,B,2023-07-14,3.00,0.02,0.671141,2.94,3.00,2.94,451000,1.343342e+03
6,SINGER,2023-07-13,2023-07-13,8.75,8.85,161377952,9.80,8.75,1.14,B,2023-07-14,9.10,0.25,2.824859,8.85,9.10,8.85,6780069,6.106198e+04
7,STA,2023-07-13,2023-07-13,16.60,17.00,8877496,17.40,16.60,2.41,B,2023-07-14,17.00,0.00,0.000000,17.00,17.20,16.90,4329851,7.368849e+04
8,SYNEX,2023-07-12,2023-07-13,12.10,11.70,1968720,12.10,11.70,-3.31,B,2023-07-14,11.80,0.10,0.854701,11.60,12.00,11.60,171838,2.037335e+03
9,TMT,2023-07-13,2023-07-13,7.25,7.30,1227936,7.50,7.25,0.69,B,2023-07-14,7.30,0.00,0.000000,7.30,7.35,7.25,33303,2.431619e+02


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,CPNREIT,2023-07-13,2023-07-13,10.90,10.80,15892330,11.00,10.70,-0.92,B,10.90,0.10,1206570,2023-07-14
1,DIF,2023-07-04,2023-07-13,11.00,10.70,561222064,11.20,10.70,-2.73,B,10.70,0.00,10919380,2023-07-14
2,GVREIT,2023-07-12,2023-07-13,8.25,8.30,1229618,8.50,8.25,0.61,B,8.40,0.10,603905,2023-07-14
3,JMT,2023-07-13,2023-07-13,33.00,34.75,91476376,37.25,33.00,5.30,B,35.50,0.75,20654942,2023-07-14
4,MCS,2023-07-06,2023-07-13,6.45,6.20,14584016,6.65,6.20,-3.88,B,6.35,0.15,135358,2023-07-14
5,SENA,2023-07-04,2023-07-13,3.00,2.98,21091856,3.02,2.94,-0.67,B,3.00,0.02,451000,2023-07-14
6,SINGER,2023-07-13,2023-07-13,8.75,8.85,161377952,9.80,8.75,1.14,B,9.10,0.25,6780069,2023-07-14
7,STA,2023-07-13,2023-07-13,16.60,17.00,8877496,17.40,16.60,2.41,B,17.00,0.00,4329851,2023-07-14
8,SYNEX,2023-07-12,2023-07-13,12.10,11.70,1968720,12.10,11.70,-3.31,B,11.80,0.10,171838,2023-07-14
9,TMT,2023-07-13,2023-07-13,7.25,7.30,1227936,7.50,7.25,0.69,B,7.30,0.00,33303,2023-07-14


In [20]:
df_merge[colu].shape

(35, 14)

In [21]:
df_merge.shape

(35, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-07-13 2023-07-14  33.00  34.00   3.03%  16,615,016  35.75  33.00 I Update
AWC      2023-07-14 2023-07-14   4.44   4.62   4.05% 534,632,172   4.94   4.44 I Insert
BBL      2023-07-14 2023-07-14 156.50 158.00   0.96% 114,348,880 161.00 155.00 I Insert
BCH      2023-07-13 2023-07-14  17.20  17.50   1.74%  30,625,096  17.90  17.20 I Update
BDMS     2023-07-14 2023-07-14  27.50  27.75   0.91% 458,309,208  28.00  26.75 O Insert
BEM      2023-07-11 2023-07-14   8.30   8.50   2.41% 427,184,736   8.60   8.25 O Update
BH       2023-07-14 2023-07-14 215.00 220.00   2.33%  26,630,760 228.00 215.00 O Insert
CPNREIT  2023-07-14 2023-07-14  10.80  10.90   0.93%  15,892,330  11.00  10.70 B Insert
DIF      2023-07-04 2023-07-14  11.00  10.70  -2.73% 561,222,064  11.20  10.70 B Update
EA       2023-07-12 2023-07-14  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb